In [111]:

from osdatahub import FeaturesAPI, Extent, NGD
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os
import numpy as np
from convertbng.util import convert_bng, convert_lonlat
from os_functions import OSparam_feature,building_height_radius
from folium.plugins import measure_control

key = os.environ.get('OS_API_KEY')
crs = "EPSG:27700"

## Line bearings to Buildings

In [2]:
X = 180700.
Y = 44777.
radius = 50
clip = True
product = 'topographic_area'

In [6]:
BuildingData = building_height_radius(X, Y, radius,product,key, clip)
BuildingGeoSeries = BuildingData['geometry'][BuildingData['Theme'] == 'Buildings']
BuildingGeoSeries = BuildingGeoSeries.to_crs(27700)
BuildingDistances = BuildingGeoSeries.distance(Point(X,Y))
LineDistances = BuildingGeoSeries.shortest_line(Point(X,Y))

In [7]:
import folium
from shapely.geometry import Point, LineString

# Define the line and start/end points
line = LineDistances.iloc[0]
end_point = Point(line.coords[0])
start_point = Point(line.coords[-1])
line_points = [p for p in line.coords]
lats = convert_lonlat(*zip(*line_points))[0]
lons = convert_lonlat(*zip(*line_points))[1]

points = []
for i in range(len(lats)):
    points.append([lons[i], lats[i]])

start_lon, start_lat = convert_lonlat(start_point.x, start_point.y)
end_lon,end_lat = convert_lonlat(end_point.x, end_point.y)

# Define the projection
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth,back_azimuth,distance = geodesic.inv(start_lon[0], start_lat[0], end_lon[0], end_lat[0])

# Create a folium map centered on the line
m = folium.Map(location=[start_lat[0],start_lon[0]], zoom_start=19)

# # Add the line to the map in blue
folium.PolyLine(points, color='blue',popup=fwd_azimuth).add_to(m)

# # Add the start point to the map in red
folium.Marker(location=[start_lat[0],start_lon[0]], icon=folium.Icon(color='green')).add_to(m)

# # Add the end point to the map in green
folium.Marker(location=[end_lat[0], end_lon[0]], icon=folium.Icon(color='red')).add_to(m)

# # Display the map
m
# points

## Road structure

In [124]:
features = FeaturesAPI(key, 'highways_road_link', extent)
results = features.query(limit=500)
if len(results['features']) == 0:
    out = 0
TA_gdf = gpd.GeoDataFrame.from_features(results['features'], crs=crs)
TA_gdf.explore()

In [34]:
ngd_collections = NGD.get_collections()

In [50]:
for i in ngd_collections['collections']:
    print(i['id'])

bld-fts-building-1
bld-fts-buildingline-1
bld-fts-buildingpart-1
gnm-fts-namedarea-1
gnm-fts-namedpoint-1
lnd-fts-land-1
lnd-fts-landform-1
lnd-fts-landformline-1
lnd-fts-landformpoint-1
lnd-fts-landpoint-1
lus-fts-site-1
lus-fts-siteaccesslocation-1
lus-fts-siteroutingpoint-1
str-fts-compoundstructure-1
str-fts-structure-1
str-fts-structureline-1
str-fts-structurepoint-1
trn-fts-cartographicraildetail-1
trn-fts-rail-1
trn-fts-roadline-1
trn-fts-roadtrackorpath-1
trn-ntwk-connectinglink-1
trn-ntwk-connectingnode-1
trn-ntwk-ferrylink-1
trn-ntwk-ferrynode-1
trn-ntwk-ferryterminal-1
trn-ntwk-path-1
trn-ntwk-pathlink-1
trn-ntwk-pathnode-1
trn-ntwk-pavementlink-1
trn-ntwk-railwaylink-1
trn-ntwk-railwaylinkset-1
trn-ntwk-railwaynode-1
trn-ntwk-road-1
trn-ntwk-roadjunction-1
trn-ntwk-roadlink-1
trn-ntwk-roadlink-2
trn-ntwk-roadnode-1
trn-ntwk-street-1
trn-rami-averageandindicativespeed-1
trn-rami-highwaydedication-1
trn-rami-maintenancearea-1
trn-rami-maintenanceline-1
trn-rami-maintenancepoi

In [103]:
X = 180700.
Y = 44777.
radius = 50
clip = True
product = 'trn-ntwk-roadlink-1'
extent = Extent.from_radius((X,Y), radius, "EPSG:27700")

In [98]:
def OSparam_ngd(u,v,rad,product,key,clip):
    extent = Extent.from_radius((u,v), rad, "EPSG:27700")
    ngd = NGD(key, product)
    results = ngd.query(max_results=500, extent=extent)
    if len(results['features']) == 0:
        out = 0
    TA_gdf = gpd.GeoDataFrame.from_features(results['features'],crs='EPSG:4326')
    return TA_gdf


In [99]:
def roads_radius(X, Y, radius,product,key, clip):
    TA = OSparam_ngd(X, Y, radius, product,key, clip)
    TA_ll = TA.to_crs(4326)
    lon, lat = convert_lonlat(X,Y)
    map = TA_ll.explore() ## colour by height
    folium.Marker([lat[0], lon[0]],popup=[lat[0],lon[0]]).add_to(map)
    return TA_ll

In [101]:
roads_radius(X, Y, radius,product,key, clip).explore()